In [ ]:
# ------------------------------------------------------------------------------------------------
# FLNSim: Simulating federated learning over LoRa networks
#
# Copyright (c) 2025 Anshika Singh and Siddhartha S. Borkotoky <siddhartha.borkotoky@gmail.com>
# All rights reserved for original contributions and modifications.
#
# This simulator includes and adapts code from the following projects:
#
# • Flower (flwr) – Federated Learning Framework
#   Copyright 2020–2025 Flower Labs GmbH (formerly ADAP/University of Oxford).
#   Licensed under the Apache License, Version 2.0.
#   Original source: https://github.com/adap/flower 
#
# • LoRaSim – Discrete-event LoRa network simulator
#   Copyright 2016–2017 Thiemo Voigt & Martin Bor.
#   Licensed under the Creative Commons Attribution 4.0 International License (CC BY 4.0).
#   Original source: https://github.com/paafam/LoRaSim  
#
# Note: Portions of this code have been modified from their original implementations.
#       We retain all original notices, copyright statements, and license texts.
#
# This work is licensed under the Creative Commons Attribution 4.0 International License. 
# To view a copy of this license, visit http://creativecommons.org/licenses/by/4.0/.
# ----------------------------------------------------------------------------------------------------


# Overview:

# The program simulates federated learning sessions (a CNN-based digit classification task on the MNIST dataset) 
# assuming that the server is connected to a LoRa gateway and the clients are LoRa end devices.  

# The simulation outputs are as follows (written to a file output.txt): 
    # the accuracy of the global model at the end of each round 
    # the round completion time, defined as the time when all sampled clients finish transmitting their local updates
    # the downlink airtime, which is the total time the server spends transmitting in a round
    # the \cumulative uplink airtime, defined as the total time all sampled clients spend transmitting in a round, aggregated across clients
  
# The input parameters are 
    # num_sessions: Number of FL sessions to simulate (final results will be averaged over the sessions)
    # rounds_per_session: Number of training rounds per session   
    # num_clients: Number of clients
    # network_radius: Network radius in meters (clients will be randomly deployed within a circle of this radius, with the server at the origin)
    # spreading_factor_FL: LoRa spreading factor to be used for transmitting FL updates
    # use_quantization: When set to True, model parameter values will be quantized
    # quantization_bits: Number of quantization bits (supports 1, 2, and 4 bits)
    # use_sparsity: When set to True, model parameters values are sparsified by setting values below a threshold to zero, and if so, what threshold to use
    # sparsity_threshold: Threshold for sparsification
    # apply_zlib: Whether to compress the updates prior to transmission
    # LoRa_Class: The class of LoRaWAN operation to be used (supports class B and C)
    # ping_slot_duration: Ping slot periodicity (only for Class B)
    # Sampling ratio (the fraction of clients chosen for update exchange in each round)
    # FEC_rate: Rate of the FEC applied to the fragments
    # duty_cycle_percentage: Maximum permitted value for transmitter duty cycle (in percentage)
    # interferer_intensity: Number of interferers per sq. meter (assumes a Poisson Point Process)
    # full_simulation: Whether to actually simulate LoRa tranmissions or apply analytical approximations
    # mean_interf_interval: Mean of the time interval (in milliseconds) between two consecutive packets from an interferer
    # interf_payload_range: Range of values for interfering frame payload (an interfering frame carries a payload that is chosen uniformly at random from within this range)             
    # LoRa_channel_code: Defines the channel coding rate for LoRa frames
    # bandwidth: Transmission bandwidth (kHz)
    # Ptx: power of the transmitted signal in dBm
    # path_loss_exponent: Models the exponential decaying of signals
    # num_channels: Number of non overlapping frequency bands available to LoRa transmitters
        


In [ ]:
!pip install tenseal
!pip install numpy
!pip install torch
!pip install torchvision
!pip install simpy

In [ ]:
!python --version

In [ ]:
import time
import statistics
import copy
import simpy
import random
import numpy as np
import math
import sys
import matplotlib.pyplot as plt
import os
from matplotlib.patches import Rectangle
import operator as op
from functools import reduce

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
!pip install scikit-learn
import sklearn

In [ ]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import tenseal as ts
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
import matplotlib.pyplot as plt
import io
import zlib
from scipy.special import gammainc
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import random
from collections import OrderedDict
from sklearn.model_selection import train_test_split
from scipy.special import gammainc, comb
from datetime import datetime
import pytz

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

In [ ]:
# Flower model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)  # 1 input channel
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(16 * 4 * 4, 120)  # MNIST input -> 28x28 -> conv/pool -> 4x4
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
from flwr_datasets import FederatedDataset
from torch.utils.data import DataLoader, random_split
from torchvision import transforms


BATCH_SIZE = 32

def load_datasets(num_clients: int):
    # Load FederatedDataset for MNIST
    fds = FederatedDataset(dataset="mnist", partitioners={"train": num_clients})

    # Standard MNIST transform
    pytorch_transforms = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])

    def apply_transforms(batch):
        batch["image"] = [pytorch_transforms(img) for img in batch["image"]]
        return batch

    trainloaders = []
    valloaders = []
    datasets = []

    # Loop over each client partition
    for cid in range(num_clients):
        partition = fds.load_partition(cid)
        partition_train_test = partition.train_test_split(test_size=0.2, seed=42)
        partition_train_test = partition_train_test.with_transform(apply_transforms)

        trainloader = DataLoader(partition_train_test["train"], batch_size=BATCH_SIZE, shuffle=True)
        valloader = DataLoader(partition_train_test["test"], batch_size=BATCH_SIZE)

        trainloaders.append(trainloader)
        valloaders.append(valloader)
        datasets.append(partition)  # Optional: Store raw partition for reference

    # Load and transform global test set
    testset = fds.load_split("test").with_transform(apply_transforms)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)

    return trainloaders, valloaders, testloader, datasets


In [ ]:
 # Global model
global_model = Net().to(DEVICE)

In [ ]:
def train(net, trainloader, accuracy, local_loss, epochs: int):
  """Train the network on the training set."""
  criterion = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(net.parameters(), lr= 0.01)
  params = list(net.parameters())
  net.train()
  for epoch in range(epochs):
      correct, total, epoch_loss = 0, 0, 0.0
      for batch in trainloader:
          images, labels = batch["image"].to(DEVICE), batch["label"].to(DEVICE)
          optimizer.zero_grad()
          outputs = net(images)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
          # Metrics
          epoch_loss += loss
          total += labels.size(0)
          correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
      epoch_loss /= len(trainloader.dataset)
      epoch_acc = correct / total
      accuracy.append(epoch_acc)
      local_loss.append(epoch_loss.item())
      #print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


In [ ]:
# Modified test code 
def test(net, testloader):
    net.eval()
    loss = 0.0
    correct = 0
    total = 0
    y_true = []
    y_pred = []
    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        for batch in testloader:
            images, labels = batch["image"].to(DEVICE), batch["label"].to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    accuracy = correct / total
    return loss, accuracy, y_true, y_pred


In [ ]:
# Helper function to pack 4-bit values into bytes
def pack_4bit(tensor):
    """Pack 4-bit values (0–15) into bytes (2 values per byte)."""
    tensor = tensor.astype(np.uint8)
    assert np.all(tensor < 16), "All values must be 4-bit (0–15)"

    packed = np.zeros((tensor.size + 1) // 2, dtype=np.uint8)

    for i in range(tensor.size):
        byte_idx = i // 2
        if i % 2 == 0:
            # Lower 4 bits
            packed[byte_idx] |= (tensor[i] & 0xF)
        else:
            # Upper 4 bits
            packed[byte_idx] |= (tensor[i] & 0xF) << 4

    return packed

# Helper function to unpack 4-bit values from bytes
def unpack_4bit(packed, original_shape):
    """Unpack 4-bit values from bytes to original shape."""
    unpacked = np.zeros(np.prod(original_shape), dtype=np.uint8)
    
    for i in range(unpacked.size):
        byte_idx = i // 2
        if i % 2 == 0:
            # Lower 4 bits
            unpacked[i] = packed[byte_idx] & 0xF
        else:
            # Upper 4 bits
            unpacked[i] = (packed[byte_idx] >> 4) & 0xF
    
    return unpacked.reshape(original_shape)



# Helper function to pack 2-bit values into bytes
def pack_2bit(tensor):
    """Pack 2-bit values (0-3) into bytes (4 values per byte)."""
    tensor = tensor.astype(np.uint8)
    packed = np.zeros((tensor.size + 3) // 4, dtype=np.uint8)
    for i in range(tensor.size):
        byte_idx = i // 4
        bit_idx = (i % 4) * 2
        packed[byte_idx] |= (tensor[i] & 0x3) << bit_idx
    return packed

# Helper function to unpack 2-bit values from bytes
def unpack_2bit(packed, original_shape):
    """Unpack 2-bit values from bytes to original shape."""
    unpacked = np.zeros(np.prod(original_shape), dtype=np.uint8)
    for i in range(unpacked.size):
        byte_idx = i // 4
        bit_idx = (i % 4) * 2
        unpacked[i] = (packed[byte_idx] >> bit_idx) & 0x3
    return unpacked.reshape(original_shape)

def pack_1bit(tensor):
    tensor = tensor.astype(np.uint8)
    packed = np.zeros((tensor.size + 7) // 8, dtype=np.uint8)
    for i in range(tensor.size):
        byte_idx = i // 8
        bit_idx = i % 8
        packed[byte_idx] |= (tensor[i] & 0x1) << bit_idx
    return packed

def unpack_1bit(packed, original_shape):
    """Unpack 1-bit values from bytes to original shape."""
    unpacked = np.zeros(np.prod(original_shape), dtype=np.uint8)
    for i in range(unpacked.size):
        byte_idx = i // 8
        bit_idx = i % 8
        unpacked[i] = (packed[byte_idx] >> bit_idx) & 0x1
    return unpacked.reshape(original_shape)

def get_model_size(model):
    """Calculate model size in KB. Assumes 4-byte representation of each non-zero parameter, with no further quantization or sparsification
    """
    # Count non-zero parameters
    non_zero_params = 0
    for param in model.parameters():
        non_zero_params += param.numel()
        
    # Each parameter has a 4-byte representation
    size_bytes = non_zero_params * 4
    
    return size_bytes / 1024  # Convert to KB


def sparsify(update_dict, threshold = 0.001):
    """ Sets parameter values below the threshold to zero """
    sparsity_levels = []
    
    for tensor in update_dict.values():
        mask = torch.abs(tensor) < threshold
        sparsity_levels.append(mask.float().mean().item())
        tensor[mask] = 0
    
    avg_sparsity = statistics.mean(sparsity_levels) if sparsity_levels else 0.0
    return update_dict, avg_sparsity


def quantize(tensor, quantization_bits = 2):
    tensor_np = tensor.cpu().numpy()
    shape = tensor_np.shape
    
    if quantization_bits in [1,2,4]:
        min_val, max_val = tensor_np.min(), tensor_np.max()
        levels = (2**quantization_bits) - 1

        if max_val == min_val:
            quantized_tensor = np.zeros_like(tensor_np, dtype=np.uint8)
        else:
            quantized_tensor = np.round((tensor_np - min_val) / (max_val - min_val) * levels).astype(np.uint8)
            
        data = quantized_tensor.flatten()

        if quantization_bits == 4:
            data = pack_4bit(data)
        elif quantization_bits == 2:
            data = pack_2bit(data)
        elif quantization_bits == 1:
            data = pack_1bit(data)
            
        data_bytes = data.tobytes()
        tensor_size = np.prod(shape) * (quantization_bits / 8)
        min_max = (min_val, max_val)
    else:
        data_bytes = tensor_np.tobytes()
        tensor_size = np.prod(shape) * 4
        min_max = None
    return data_bytes, shape, min_max, tensor_size
    

def compress_update(model, reference_model, use_sparsity, use_quantization, apply_zlib, sparsity_threshold, quantization_bits):
    """Compress model update (delta) dictionary"""
    compressed_data = {}
    original_shapes = {}
    original_min_max = {}
    uncompressed_size = 0

    model_dict = {}
    for name, param in model.named_parameters():
        model_param = next(p for n, p in model.named_parameters() if n == name)
        if reference_model is None:
            model_dict[name] = model_param.data
        else:
            ref_model_param = next(p for n, p in reference_model.named_parameters() if n == name)
            model_dict[name] = model_param.data - ref_model_param.data

    if use_sparsity:
        model_dict, sparsity = sparsify(model_dict, threshold = sparsity_threshold)
    
    for key, tensor in model_dict.items():
        data_bytes, shape, min_max, tensor_size = quantize(tensor, quantization_bits if use_quantization else 0)
        original_shapes[key] = shape
        if min_max is not None:
            original_min_max[key] = min_max
        uncompressed_size += tensor_size
        
        if apply_zlib:
            compressed_data[key] = zlib.compress(data_bytes, level = 9)
        else:
            compressed_data[key] = data_bytes
    
    compressed_size = sum(len(v) for v in compressed_data.values()) / 1024
    compression_ratio = uncompressed_size / (compressed_size * 1024) if compressed_size > 0 and apply_zlib else 1.0
    
    return compressed_data, compressed_size, original_shapes, original_min_max, compression_ratio

def decompress_update(compressed_data, original_shapes, original_min_max, use_quantization=False, apply_zlib=False, quantization_bits=2):
    """Decompress model update (delta) dictionary"""
    update_dict = {}
    for key, data in compressed_data.items():
        data_bytes = zlib.decompress(data) if apply_zlib else data
        tensor = dequantize(data_bytes, original_shapes[key], original_min_max.get(key), quantization_bits if use_quantization else 0)
        update_dict[key] = tensor
    return update_dict


def dequantize(data_bytes, shape, min_max, quantization_bits=0):
    expected_elements = np.prod(shape)

    if quantization_bits in [1,2,4]:
        if quantization_bits == 2:
            expected_bytes = (expected_elements + 3) // 4
            if len(data_bytes) != expected_bytes:
                raise ValueError(f"Buffer size mismatch for {key}: got {len(data_bytes)} bytes, expected {expected_bytes}")
            packed_tensor = np.frombuffer(data_bytes, dtype=np.uint8)
            quantized_tensor = unpack_2bit(packed_tensor, shape)
        elif quantization_bits == 1:
            expected_bytes = (expected_elements + 7) // 8
            if len(data_bytes) != expected_bytes:
                raise ValueError(f"Buffer size mismatch for {key}: got {len(data_bytes)} bytes, expected {expected_bytes}")
            packed_tensor = np.frombuffer(data_bytes, dtype=np.uint8)
            quantized_tensor = unpack_1bit(packed_tensor, shape)
        else:
            expected_bytes = (expected_elements + 1) // 2
            if len(data_bytes) != expected_bytes:
                raise ValueError(f"Buffer size mismatch for {key}: got {len(data_bytes)} bytes, expected {expected_bytes}")
            packed_tensor = np.frombuffer(data_bytes, dtype=np.uint8)
            quantized_tensor = unpack_4bit(packed_tensor, shape)
        
        if min_max is not None:
            min_val, max_val = min_max
            levels = (2 ** quantization_bits) - 1
            if max_val == min_val:
                tensor_np = np.full(shape, min_val, dtype=np.float32)
            else:
                tensor_np = (quantized_tensor / levels) * (max_val - min_val) + min_val
        else:
            tensor_np = quantized_tensor.astype(np.float32)
    else:
        expected_bytes = expected_elements * 4
        if len(data_bytes) != expected_bytes:
            raise ValueError(f"Buffer size mismatch for {key}: got {len(data_bytes)} bytes, expected {expected_bytes}")
        tensor_np = np.frombuffer(data_bytes, dtype=np.float32).reshape(shape)
    
    return torch.tensor(tensor_np, dtype=torch.float32)

In [ ]:
# For a frame/fragment delivery probability Pd, this function evaluates the probability that k out of n frames are successully delivered
import math
from scipy.special import gammainc
from scipy.stats import binom, poisson

def success_prob_k_of_n(Pd, k, n):                          
    
    j = np.arange(k, n + 1)

    # Use binomial distribution
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        MDP = np.sum(comb(n,j)*((Pd**j)*((1-Pd)**(n-j))))

    # If the binomial distribution evaluates to NaN, try the Poisson approximation
    if math.isnan(MDP):
        if Pd < 0.5:
            MDP = np.sum(poisson.pmf(j, n * Pd))
        else:
            j1 = np.arange(0, n-k+1)
            MDP = np.sum(poisson.pmf(j1, n * (1-Pd)))
            
    return MDP

# LoRa MTU (Maximum Transmission Unit)    
def LoRa_MTU(sf):                                             
    return 222 if sf in [7, 8] else 115 if sf == 9 else 51  # For SF 10, 11, 12


# Frame length computation functions
def LoRa_frame_duration(sf, payload_bytes):
    npr = 8
    h = 1  # Optional header
    c = 1  # Coding rate
    bw = 125 * 10**3  # Bandwidth in Hz
    return ((npr + 4.25) * (2**sf / bw) +
            (8 + max(np.ceil((2 * payload_bytes - sf - 5 * h + 11) / (sf - 2*(sf >= 11))) * (c + 4), 0)) * (2**sf / bw))

In [ ]:
# This function computes the probability that a frame/fragment transmitted over a 
# given distance is succesfully delivered
# Analysis based on: S. S. Borkotoky, "Balancing the Energy Consumption and Latency of Over-the-Air Firmware Updates in LoRaWAN," in IEEE Transactions on Industrial Informatics, doi: 10.1109/TII.2025.3563539.

import numpy as np
from scipy.integrate import trapz

def compute_fragment_delivery_prob(receiver_distances, sf, fragment_size):
    
    
    # Inteference characterstics
    mean_interval = mean_interf_interval  # Mean interval between two consecutive frames from an interferer (ms)
    avg_interferer_payload = int(np.mean(interf_payload_range))  # Payload per interferer message (bytes)
    lambda_interf = 1 / mean_interval  # Interfering frames per millisecond
        
    # Result storage for model delivery probabilities
    frag_delivery_probs = []

    # Iterate over each receiver
    for dist in receiver_distances:
        
        # Adjust payload size based on MTU
        payload_size = fragment_size

        # Compute frame success probability
        a_vals = np.arange(0.005, 10.005, 0.001)  # Fading coefficient range
        FSP_cond_a = np.zeros_like(a_vals)

        for j, a in enumerate(a_vals):
            f1 = 0
            for sf_interf in range(7, 13):
                xi = 10**(0.1 * cap_thresh[sf - 7, sf_interf - 7])
                des_frame_length = airtime(sf, 1, payload_size, 125)
                interf_frame_length = airtime(sf_interf, 1, avg_interferer_payload, 125)              
                vuln_wind = des_frame_length + interf_frame_length
                c_ij = (lambda_interf / num_channels) * vuln_wind
                beta_j = xi**(-1) * a * dist**(-path_loss_exp)
                gammadiff = gammainc(2 / path_loss_exp, beta_j * (simulation_radius**path_loss_exp))  #  path_loss_exp function
                f1 += c_ij * gammadiff / (beta_j**(2 / path_loss_exp))

            QI = (1 / 6) * (2 / (path_loss_exp * simulation_radius**2)) * f1
            succ_interference = (1 - QI)**avg_num_interferers

            fsensitivity_dBm = sensitivity[sf - 7]
            rx_pow_dBm = Ptx - Lpld0 + GtGr + 10 * np.log10(a) - 10 * path_loss_exp * np.log10(dist / d0)
            succ_fading = rx_pow_dBm >= fsensitivity_dBm

            FSP_cond_a[j] = succ_fading * succ_interference * np.exp(-a)

        
        P_frag = trapz(FSP_cond_a, a_vals)
        #Probability of receiving at least k out of n fragments
        frag_delivery_probs.append(P_frag)
        # print(P_frag)

    return frag_delivery_probs


In [ ]:
# This function computes the probability that an update is successfully transfered to the recipient(s)

def compute_model_delivery_prob(transmitter_coordinates, receiver_coordinates, sf, frag_size, k, r):
    
    model_delivery_probs = []
    receiver_distances = []

    for rx_coordinates in receiver_coordinates:
        dist = np.sqrt((transmitter_coordinates[0] - rx_coordinates[0])**2 + (transmitter_coordinates[1] - rx_coordinates[1])**2)
        receiver_distances.append(dist)
    
    fragment_delivery_probs = compute_fragment_delivery_prob(receiver_distances, sf, frag_size)

    #print("fragment_delivery_probs, receiver_distances, sf, frag_size: ",fragment_delivery_probs, receiver_distances, sf, frag_size)
    
    n = k + r

    for P_frag in fragment_delivery_probs:
        P_model = success_prob_k_of_n(P_frag, k, n)
        model_delivery_probs.append(P_model)

    return model_delivery_probs

In [ ]:
from collections import OrderedDict
import torch
from typing import List, Union
import numpy as np
import torch.nn as nn

def set_parameters_to_clients(net: nn.Module, parameters: Union[List[np.ndarray], nn.Module]):
    """Set the parameters of net to the provided parameters or model."""
    if isinstance(parameters, nn.Module):
        parameters = [val.cpu().numpy() for _, val in parameters.state_dict().items()]
    
    # Check if model has parameters
    try:
        device = next(net.parameters()).device
    except StopIteration:
        raise ValueError("Model has no parameters to set.")
    
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v).to(device) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

In [ ]:
def aggregate_updates(updates):
    """Average decompressed updates"""
    if not updates:
        return {}
    
    aggregated = {}
    for key in updates[0].keys():
        tensors = [update[key] for update in updates]
        aggregated[key] = torch.stack(tensors).mean(dim=0)
    
    return aggregated

In [ ]:
#
# check for collisions at base station
# Note: called before a packet (or rather node) is inserted into the list
#

def checkcollision(packet):
    col = 0 # flag needed since there might be several collisions for a packet
    # lost frames don't collide
    # (they go undetected since the received power is below the receiver sensitivity)
    if packet.detection_failure:
        return 0
    if packetsAtDest[packet.rcvr_id]:
        # see if another frame currently being recieved is capable of causing the loss of the frame
        # do the above for every other frame that are currently being received
        for other in packetsAtDest[packet.rcvr_id]:
            if other.id != packet.nodeid:
                if packet.freq == other.packet[packet.rcvr_id].freq:                   
                    col = timingandpowercheck(packet, other.packet[packet.rcvr_id])                        
        return col
    return 0

In [ ]:
#
# this function checks if the freshly arrived frame (p1) is lost due to a frame p2 that
# was already being received by the gateway. Returns 1 if frame loss conditions are satisfied
#
def timingandpowercheck(p1, p2):
    #
    # Is p1 (which just arrived) lost due to p2 (which arrived earlier)?
    #
    
    # Capture-effect threshold 
    powerThreshold = cap_thresh[p1.sf - 7, p2.sf - 7] # dB

    # no. of preamble symbols in a frame
    Npream = 8

    # we can lose at most (Npream - 5) * Tsym of p1's preamble
    Tpreamb = 2**p1.sf/(1.0*p1.bw) * (Npream - 5)

    # the time at which p2 will end
    p2_end = p2.addTime + p2.rectime

    # ending time for the critical section of p1
    p1_cs = env.now + Tpreamb

    # Is p2 a strong inteferer from p1's perspective?    
    p2_is_strong_interferer = (p1.rssi - p2.rssi) < powerThreshold

    # check whether p2 corrupts p1's critical section
    if p2_is_strong_interferer and p1_cs < p2_end:
        # p1 was lost
        p1.collided = 1
            
    #
    # Is p2 lost due to p1?
    #
    
    # Is p1 a strong inteferer from p2's perspective?    
    p1_is_strong_interferer = (p2.rssi - p1.rssi) < powerThreshold

    # If p1 is a strong interferer, then p2 is lost
    if p1_is_strong_interferer:        
        p2.collided = 1
                
    return p1.collided

In [ ]:
#
# this function computes the airtime of a packet
# according to LoraDesignGuide_STD.pdf
#

def airtime(sf,cr,pl,bw):
    H = 1        # implicit header disabled (H=0) or not (H=1)
    DE = 0       # low data rate optimization enabled (=1) or not (=0)
    Npream = 8   # number of preamble symbol (12.25  from Utz paper)

    if bw == 125 and sf in [11, 12]:
        # low data rate optimization mandated for BW125 with SF11 and SF12
        DE = 1
    if sf == 6:
        # can only have implicit header with SF6
        H = 1

    Tsym = (2.0**sf)/bw
    Tpream = (Npream + 4.25)*Tsym
    payloadSymbNB = 8 + max(math.ceil((8.0*pl-4.0*sf+28+16-20*H)/(4.0*(sf-2*DE)))*(cr+4),0)
    Tpayload = payloadSymbNB * Tsym
    return Tpream + Tpayload

In [ ]:
#
# this function creates a receiver
#
class myReceiver():
    def __init__(self, id, coordinates):
        
        self.id = id

        self.x = coordinates[0]
        self.y = coordinates[1]

            
        # Buffer contents at Dest
        self.buffered_node = []
        self.buffered_seq = []

        # To keep track of the number of FL fragments received
        self.total_rcvd = 0 

In [ ]:
#
# this function creates a node
#

class myTransmitter():
    def __init__(self, id, coordinates, mean_interf_interval):
        global rx_nodes, spreading_factor_FL

        self.id = id
        self.period = mean_interf_interval
        self.tx_type = 1
        self.packet = []
        self.frame_seqnum = -1


        # node 0 is the FL transmitter, others are interferers
        if self.id == 0:
            self.tx_type = 0 # Periodic traffic
            self.period = 0 # Send frames one after the other
            self.sf = spreading_factor_FL # Use fixed SF
            self.CU = 0 # Do not request ACKs 
            self.x = coordinates[0]
            self.y = coordinates[1]  
        else:
            self.tx_type = 1 # Exponential traffic
            self.period = mean_interf_interval
            self.sf = np.random.randint(7, 13) # Use a random SF
                        
            self.CU = 0 # Do not request ACKs 
            self.x = coordinates[0]
            self.y = coordinates[1]      
        
        # intialize counters to keep track of how many messages a node has sent
        # and how many have been successfully delivered
        self.sent = 0
        self.delivered = 0

In [ ]:
#
# this function creates a packet (associated with a node)

class myPacket():
    def __init__(self, nodeid, sf, plen, distance, rcvr_id):
        global Ptx
        global path_loss_exp
        global d0
        global Lpld0
        global LoRa_channel_code
        global bandwidth


        # new: base station ID
        self.rcvr_id = rcvr_id

        # originating node id
        self.nodeid = nodeid

        # spreading factor, code rate, and bandwidth for the frame
        self.sf = sf
        self.cr = LoRa_channel_code
        self.bw = bandwidth

        # payload length
        self.pl = plen
     
        # propagation loss for the frame
        propagation_loss = Lpld0 + 10*path_loss_exp*math.log10(distance/d0)

        # fading gain for the frame (Rayleigh fading, equivalent to Nakagami-m with m = 1)
        nakagami_m = 1  
        fading_gain = 10 * math.log10(random.gammavariate(nakagami_m, 1.0/float(nakagami_m)))

        # received power in the frame
        self.rssi = Ptx + GtGr - propagation_loss + fading_gain

        # choose one of the three center frquencies at random
        self.freq = random.randint(1, num_channels)

        # on-air time for the frame
        self.rectime = airtime(self.sf,self.cr,self.pl,self.bw)
        
        
        # flags for collision and detection failure
        self.collided = 0
        self.processed = 0 
        self.detection_failure = 0        

        if self.rssi < sensitivity[self.sf - 7]:            
            self.detection_failure = 1

In [ ]:
#
# transmission event loop, runs for each node
# a global list of packet being processed at the gateway is maintained
#

def transmit(env, node, FL_fragment_size, interf_payload_range, tx_limit, numReceivers, rx_nodes, stop_event):
    while True:

        global packetSeq

        # payload size for the frame 
        if node.id == 0:
            payload_size = FL_fragment_size
        else:
            payload_size = random.randrange(interf_payload_range[0], interf_payload_range[1])


        # Node transmissions  
        if node.tx_type == 1:
            yield env.timeout(random.expovariate(1.0/float(node.period)))
        else:	        
            if node.sent == 0:
                yield env.timeout(random.uniform(0, node.period))
            yield env.timeout(airtime(node.sf, LoRa_channel_code, payload_size, bandwidth))

        # what will be the ending time of the current packet?
        pkt_end_time = env.now + airtime(node.sf, LoRa_channel_code, payload_size, bandwidth);
                     
        # increment the send counter for the node
        node.sent = node.sent + 1
        
        # increment the global sequence number for the frame
        packetSeq = packetSeq + 1

        # increment the local sequence number for the frame at the sending node
        node.frame_seqnum = node.frame_seqnum + 1        

        # create the packet
        node.packet = []
        
        for rcvr_id in range(0, numReceivers):
            dist = np.sqrt((node.x - rx_nodes[rcvr_id].x)*(node.x - rx_nodes[rcvr_id].x) + (node.y - rx_nodes[rcvr_id].y)*(node.y - rx_nodes[rcvr_id].y))
            node.packet.append(myPacket(node.id, node.sf, payload_size, dist, rcvr_id))
                   
        
        # packet reception at gateway    
        for rcvr_id in range(0, numReceivers):               
            if (node in packetsAtDest[rcvr_id]):
                print ("ERROR: packet already in")
            else:
                # adding packet if no collision
                collision_flag = checkcollision(node.packet[rcvr_id])
                packetsAtDest[rcvr_id].append(node)
                node.packet[rcvr_id].addTime = env.now
                node.packet[rcvr_id].seqNr = packetSeq

        # packet reception time
        yield env.timeout(airtime(node.sf, LoRa_channel_code, payload_size, bandwidth))
                
        # if packet did not collide, add it to the list of received packets
        # unless it is already in
        for rcvr_id in range(0, numReceivers):
            if not node.packet[rcvr_id].detection_failure:
                if node.packet[rcvr_id].collided == 0:
                    if (rcvr_id == 0):
                        node.delivered = node.delivered + 1
                    
                    if node.id == 0:
                        rx_nodes[rcvr_id].total_rcvd = rx_nodes[rcvr_id].total_rcvd + 1
                    
                    packetsRecDest[rcvr_id].append(node.packet[rcvr_id].seqNr)
                                                    
                    if (recPackets):
                        if (recPackets[-1] != node.packet[rcvr_id].seqNr):
                            recPackets.append(node.packet[rcvr_id].seqNr)
                    else:
                        recPackets.append(node.packet[rcvr_id].seqNr)
                                        
        # complete packet has been received by base station
        # can remove it
        for rcvr_id in range(0, numReceivers):
            if (node in packetsAtDest[rcvr_id]):
                packetsAtDest[rcvr_id].remove(node)
                # reset the packet
                node.packet[rcvr_id].collided = 0
                node.packet[rcvr_id].processed = 0

        # Stop the session once all K+R fragments have been transmitted
        if transmit_nodes[0].sent == tx_limit:
            stop_event.succeed()  # This will stop env.run()
            return        
        
        yield env.timeout(1)

In [ ]:
#
# This function simulates the transfer of FL updates over a wireless link
#

def simulate_update_transfer(transmitter_coordinates, receiver_coordinates, interferer_coordinates, sf, frag_size, k, r, isdownlink):

    # global stuff
    global transmit_nodes, packetsAtDest, packetsRecDest, recPackets, lostPackets, packetSeq, Dest_start, rx_nodes
    global env
    global total_tx_all, total_rx_all
    
    nrNodes = len(interferer_coordinates) + 1  # No. of transmitters in the network (incldues the interferers and the device transmitting the FL update)
    
    if isdownlink:
        numReceivers = len(receiver_coordinates)
    else:
        numReceivers = 1
            
    if full_simulation:
        
        transmit_nodes = []
        packetsAtDest = []
        env = simpy.Environment()
        Dest_start = []
        
        packetSeq = 0 # Will be incremented by 1 whenever a frame is sent in the network 
        
        # list of received packets
        recPackets=[]
        lostPackets = []
        
        # list of base stations 
        rx_nodes = []
        
        # list of packets at each base station, init with 0 packets
        packetsAtDest = []
        packetsRecDest = []
        
        # activate the gateways and the relays
        for i in range(0, numReceivers):
            b = myReceiver(i, receiver_coordinates[i])
            rx_nodes.append(b)
            packetsAtDest.append([])
            packetsRecDest.append([])
            Dest_start.append(1)

        # The stop_event is to be triggered once the FL update sender transmits the necessary frames (i.e., k+r fragments)
        stop_event = env.event()
        
        # activate the transmitters (FL update sender + interferers)
        for i in range(0, nrNodes):
            if i == 0:
                coordinates = transmitter_coordinates
            else:
                coordinates = interferer_coordinates[i-1]
            node = myTransmitter(i, coordinates, mean_interf_interval)
            transmit_nodes.append(node)
            env.process(transmit(env, node, frag_size, interf_payload_range, k + r, numReceivers, rx_nodes, stop_event))
            
        env.run(until = stop_event)
    
        success = []

        # Keeping track of total FL frames sent across all devices (for debugging purposes)
        total_tx_all += transmit_nodes[0].sent
        total_rx_all += rx_nodes[0].total_rcvd
        
        for rcvr_id in range(numReceivers):
            dist = np.sqrt((transmit_nodes[0].x-rx_nodes[rcvr_id].x)*(transmit_nodes[0].x-rx_nodes[rcvr_id].x)+(transmit_nodes[0].y-rx_nodes[rcvr_id].y)*(transmit_nodes[0].y-rx_nodes[rcvr_id].y))
            #print(dist, transmit_nodes[0].sent, rx_nodes[rcvr_id].total_rcvd, float(total_rx_all)/float(total_tx_all), 'sf: ', sf, 'frag_size: ', frag_size, 'k: ', k, 'r: ', r, 'downlink: ',isdownlink)
            #print("analytical: ",compute_fragment_delivery_prob([dist], sf, frag_size))
            #print("fl_pkts: ",transmit_nodes[0].sent)
            #print("interf_pkts: ",transmit_nodes[1].sent)
            success.append(rx_nodes[rcvr_id].total_rcvd >= k)
    
    else:
        success = []
        mdp = compute_model_delivery_prob(transmitter_coordinates, receiver_coordinates, sf, frag_size, k, r)

        for rcvr_id in range(numReceivers):                
            r_val = np.random.rand()
            success.append(r_val <= mdp[rcvr_id])
            
    return success
        

In [ ]:
#
# Simulate a federated learning session based on parameters specified in the main() function
#

import random
def start_simulation(session_num, total_rounds, num_clients, network_radius, duty_cycle_percentage, sampling_ratio, use_quantization, use_sparsity, sparsity_threshold, apply_zlib, quantization_bits, ping_slot_duration, spreading_factor_FL, FEC_rate, LoRa_Class, trainloaders, testloader):
    
    global global_accuracy, global_loss, prediction_accuracy, prediction_loss, precision, recall, f1
    global simulation_radius, num_interferers, avg_num_interferers
    
    compress_global_update = True
    
    compression_ratios = []
    sparsity_levels = []
    active_clients_indices = []
    client_update_sizes = []                #Track size of each clients update
    total_uplink_airtime = []               #Track total communication cost on the uplink
    round_completion_time = [];
    total_client_updates_delivered = [];
    client_distances = [];
    client_coordinates = [];
    interferer_distances = [];
    interferer_coordinates = [];
    
    downlink_airtime = [0] * total_rounds
    slotted_downlink_airtime = [0] * total_rounds 
    max_uplink_airtime = [0] * total_rounds 
    downlink_start_time = [0] * total_rounds
    downlink_end_time = [0] * total_rounds
    uplink_start_time = [0] * total_rounds
    uplink_end_time = [0] * total_rounds

    
    processing_delay = 0 # time (in seconds) needed by a client to perform a local update
    
    print("\n---------------------------\n    Starting Session",  session_num, "    \n---------------------------")

    # Assign random locations to the clients (uniform distribution over a circle)
    for client_idx in range(num_clients):
        r1 = random.uniform(0, 1)
        dist = math.sqrt(r1) * network_radius                
        theta = random.uniform(0, 2*3.14)        
        x_coord = dist * math.cos(theta)
        y_coord = dist * math.sin(theta) 
        client_distances.append(dist)
        client_coordinates.append([x_coord, y_coord])
        
    #print('Client distances from server (in meters):', client_distances) 


    # Generate a random number of interferers according to a Poisson Point Process
    interference_radius = 3000
    simulation_radius = network_radius + interference_radius
    avg_num_interferers = np.pi * (simulation_radius**2) * interferer_intensity
    
    num_interferers = np.random.poisson(avg_num_interferers)
    
    for interf_idx in range(num_interferers):
        
        # Assign interferer locations (uniform distribution over a circle)
        r1 = random.uniform(0, 1)
        dist = math.sqrt(r1) * simulation_radius        
        
        theta = random.uniform(0, 2*3.14)        
        x_coord = dist * math.cos(theta)
        y_coord = dist * math.sin(theta) 
        interferer_distances.append(dist)
        interferer_coordinates.append([x_coord, y_coord]) 

    # The gateway is at the center
    gateway_coordinates = [0,0]
    
    # Initialize models
    try:
        global_model = Net().to(DEVICE)
        client_models = [Net().to(DEVICE) for _ in range(num_clients)]
        global_model_at_client = Net().to(DEVICE)
        
        # Initialize random number generators
        torch.manual_seed(int(time.time()))
        np.random.seed(int(time.time()))
        
        for model in [global_model] + [global_model_at_client]:
            for param in model.parameters():
                if param.requires_grad:
                    if len(param.data.shape) >= 2:  # Weights (2D or higher)
                        torch.nn.init.xavier_uniform_(param.data)
                    else:  # Biases (1D)
                        torch.nn.init.uniform_(param.data, a=-0.1, b=0.1)


    except NameError as e:
        print(f"Error: 'Net' or 'DEVICE' not defined. Ensure model and device are set up: {e}")
        return
    except Exception as e:
        print(f"Error initializing models: {e}")
        traceback.print_exc()
        return
            
    for round in range(total_rounds):
        
        # Randomly sample a subset of clients for participation in this round
        subset_size = np.ceil(sampling_ratio * num_clients)
        sampled_clients = random.sample(range(num_clients), int(subset_size));
        sampled_clients.sort()
        
        # Variables to store total on-air times and the local update-delivery statistics in this round
        uplink_airtime = 0
        per_client_uplink_airtime = [0] * num_clients # Total airtime spent by the clients in this session
        
        total_updates_delivered = 0
        
        accuracy = []
        local_loss = []
        round_sizes= []
        round_sparsity= []
        round_compression= []

        compressed_updates= []
        shapes_list= []
        min_max_list= []

        clients_to_tx = [] # Clients that are going to transmit their local updates to the server in this round
        local_updates_delivered = [] # Clients that successfully delivered their local updates to the master in this round
        
        if round == 0:
            # In the intial round, every client has the same random model parameters
            # Global model assumed to be generated locally at each device (using same random seed, so that everyone has the same model)
            # No downlink transmissions needed
            updated_clients = range(num_clients)
            global_model_at_client = copy.deepcopy(global_model)
            downlink_airtime[round] = 0 

            # Although the model is not actually transmitted, server still computes how long it will take to transmit it on the uplink
            # The result will be used to calculate when the next round should start such that duty cycle restrictions are satsified
            if compress_global_update:
                compressed_global_update, compressed_global_update_size, shapes, min_max, comp_ratio = compress_update(global_model, None, use_sparsity, use_quantization, apply_zlib, sparsity_threshold, quantization_bits)
                update_size_bytes = compressed_global_update_size * 1024  # convert kilobytes to bytes
            else:
                update_size_bytes = get_model_size(global_model) * 1024
            
            frag_size = LoRa_MTU(spreading_factor_FL)
            k0 = np.ceil(float(update_size_bytes) / float(frag_size))
            r0 = np.ceil(float(k0) / float(FEC_rate)) - k0 
            airtime0 = (k0 + r0) * LoRa_frame_duration(spreading_factor_FL, frag_size)
        
        else:
            if compress_global_update:
                                
                compressed_global_update, compressed_global_update_size, shapes, min_max, comp_ratio = compress_update(global_model, None, use_sparsity, use_quantization, apply_zlib, sparsity_threshold, quantization_bits)
                downlink_update_size_bytes = compressed_global_update_size * 1024  # convert kilobytes to bytes
    
                # The decompressed version of the global model that the client will retrieve
                update_dicts_global = []
                for i, (comp_update, shapes, min_max) in enumerate(zip([compressed_global_update], [shapes], [min_max])):
                    update_dict_global = decompress_update(compressed_global_update, shapes, min_max, use_quantization = use_quantization, apply_zlib = apply_zlib, quantization_bits = quantization_bits)
                    update_dicts_global.append(update_dict_global)
                aggregated_global_update = aggregate_updates(update_dicts_global)
                with torch.no_grad():
                    for name, param in global_model_at_client.named_parameters():
                        if name in aggregated_global_update:
                            param.data.copy_(aggregated_global_update[name].to(param.device))
    
            else:
                global_model_at_client = copy.deepcopy(global_model) # No compression; so client get the exact same model as the server
                downlink_update_size_bytes = get_model_size(global_model) * 1024 # convert kilobytes to bytes
    
                
            # Transmission parameters to be used for server-to-client communications
            downlink_sf = spreading_factor_FL
            downlink_frag_size = LoRa_MTU(downlink_sf)
            downlink_k = np.ceil(float(downlink_update_size_bytes) / float(downlink_frag_size))
            downlink_r = np.ceil(float(downlink_k) / float(FEC_rate)) - downlink_k 
                    
            # Total airtime incurred on the downlink in this round
            downlink_airtime[round] = (downlink_k + downlink_r) * LoRa_frame_duration(downlink_sf, downlink_frag_size)

            # Find the clients that received the global update
            sampled_client_coordinates = [] 
            updated_clients = []
            
            for client_no in sampled_clients:
                sampled_client_coordinates.append(client_coordinates[client_no])
                
            global_model_delivered = simulate_update_transfer(gateway_coordinates, sampled_client_coordinates, interferer_coordinates, downlink_sf, downlink_frag_size, downlink_k, downlink_r, isdownlink = True)
            for i in range(len(sampled_clients)):     
                if global_model_delivered[i]:            
                    updated_clients.append(sampled_clients[i])

            #print("Sampled clients: ", sampled_clients)
            #print("Global update delivered to: ", updated_clients)
            
        # The exisiting local models which are to be updated
        local_models_to_update = [client_models[i] for i in updated_clients if i < len(client_models)]

        # Clients that will transmit local model in this round
        for i in range(num_clients):
            if i in sampled_clients and i in updated_clients:
                clients_to_tx.append(i)

        # Downlink transmission time in this round
        if LoRa_Class == 'B':
            # Total no. of ping slots occupied by the downlink transmission multiplied by ping slot duration  
            slotted_downlink_airtime[round] = np.ceil(float(downlink_airtime[round]) / float(ping_slot_duration))  * ping_slot_duration

        # Total time that has elapsed from the session's beginning till this point (i.e., the end of this downlink transmission)
        if round == 0:
            # In round 0, there was no downlink transmission, so time needed
            downlink_start_time[round] = 0
            downlink_end_time[round] = 0
        
        elif round == 1:
            # Assumptions: 
            # All clients transmit simultaneously  --> Round 0 ended as soon as the longest uplink transfer ended
            # We will wait till time t0, such that longest_uplink_transfer/t0 < duty_cycle            
            if LoRa_Class == 'C':
                downlink_start_time[round] = airtime0 * 100.0 / float(duty_cycle_percentage)
                downlink_end_time[round] = downlink_start_time[round] + downlink_airtime[round]
            else:
                slots = np.ceil(float(airtime0) / float(ping_slot_duration))
                downlink_start_time[round] = slots * ping_slot_duration * 100.0 / float(duty_cycle_percentage)
                downlink_end_time[round] = downlink_start_time[round] + slotted_downlink_airtime[round]
        
        else:
            if LoRa_Class == 'C':
                downlink_start_time[round] = downlink_start_time[round - 1] + downlink_airtime[round - 1] * 100.0 / float(duty_cycle_percentage)
                downlink_end_time[round] = downlink_start_time[round] + downlink_airtime[round]
            else:
                downlink_start_time[round] = downlink_start_time[round - 1] + slotted_downlink_airtime[round - 1] * 100.0 / float(duty_cycle_percentage)
                downlink_end_time[round] = downlink_start_time[round] + slotted_downlink_airtime[round]
                        
        
        if not updated_clients:
            print(f"\nSkipping aggregation in Round {round} as no client received global update.\n")

            prediction_accuracy.append(prediction_accuracy[-1])
            prediction_loss.append(prediction_loss[-1])
            precision.append(precision[-1])
            recall.append(recall[-1])
            f1.append(f1[-1])
            total_uplink_airtime.append(0)
            round_completion_time.append(downlink_end_time[round])
            total_client_updates_delivered.append(0)
            
            continue
        
        for client_idx, model in zip(updated_clients, local_models_to_update):
            
            set_parameters_to_clients(model, global_model_at_client)      # The client now has the latest global model parameters
            train(model, trainloaders[client_models.index(model)], accuracy, local_loss, epochs = 1)  # Train client model

            if client_idx in clients_to_tx:
                
                # Sparsify, quantize, and compress the update
                compressed_update, update_size, shapes, min_max, comp_ratio = compress_update(model, global_model_at_client, use_sparsity, use_quantization, apply_zlib, sparsity_threshold, quantization_bits)
                update_size_bytes = update_size * 1024
                
                # Spreading factor, fragment size, and redundancy allocation
                uplink_sf = spreading_factor_FL
                uplink_frag_size = LoRa_MTU(uplink_sf)
                uplink_k = np.ceil(float(update_size_bytes) / float(uplink_frag_size))
                uplink_r = np.ceil(float(uplink_k) / float(FEC_rate)) - uplink_k 
    
                UL_transfer_success = simulate_update_transfer(client_coordinates[client_idx], [gateway_coordinates], interferer_coordinates, uplink_sf, uplink_frag_size, uplink_k, uplink_r, isdownlink = False)
                
                if UL_transfer_success[0]:
                    uplink_update_delivered = True
                    total_updates_delivered += 1
                    local_updates_delivered.append(client_idx)
                else:
                    uplink_update_delivered = False
                            
                # Increment communication cost on the uplink
                uplink_airtime += (uplink_k + uplink_r) * LoRa_frame_duration(uplink_sf, uplink_frag_size)

                per_client_uplink_airtime[client_idx] = (uplink_k + uplink_r) * LoRa_frame_duration(uplink_sf, uplink_frag_size)
                
                if uplink_update_delivered:
                    compressed_updates.append(compressed_update)
                    shapes_list.append(shapes)
                    min_max_list.append(min_max if use_quantization else {})
                    round_sizes.append(update_size)
                    round_compression.append(comp_ratio)

        max_uplink_airtime[round] = np.max(per_client_uplink_airtime)
        
        # Update the elapsed time 
        if LoRa_Class == 'C':
            uplink_start_time[round] = downlink_end_time[round] + processing_delay
            uplink_end_time[round] = uplink_start_time[round] + max_uplink_airtime[round]
        else:
            uplink_start_time[round] = downlink_end_time[round] + np.ceil(float(processing_delay) / float(ping_slot_duration)) * ping_slot_duration
            uplink_end_time[round] = uplink_start_time[round] + np.ceil(float(max_uplink_airtime[round]) / float(ping_slot_duration)) * ping_slot_duration
        
        #print("Local updates received from: ", local_updates_delivered)
        
        client_update_sizes.append(round_sizes)
        sparsity_levels.append(statistics.mean(round_sparsity) if round_sparsity else 0.0)
        compression_ratios.append(statistics.mean(round_compression) if round_compression else 1.0)
        
        global_accuracy.append(statistics.mean(accuracy) if accuracy else 0.0)
        global_loss.append(statistics.mean(local_loss) if local_loss else 0.0)
        
        # Decompress updates
        update_dicts = []
        for i, (comp_update, shapes, min_max) in enumerate(zip(compressed_updates, shapes_list, min_max_list)):
            update_dict = decompress_update(comp_update, shapes, min_max, use_quantization = use_quantization, apply_zlib = apply_zlib, quantization_bits = quantization_bits)
            update_dicts.append(update_dict)
    

        # Aggregate updates
        aggregated_update = aggregate_updates(update_dicts)

        # Updates the global model
        with torch.no_grad():
            for name, param in global_model.named_parameters():
                if name in aggregated_update:
                    param.data += aggregated_update[name].to(param.device)
        
        # Test global model
        pred_loss, pred_accuracy, y_true, y_pred = test(global_model, testloader)
        prediction_accuracy.append(pred_accuracy)
        prediction_loss.append(pred_loss)
        prec = precision_score(y_true, y_pred, average='macro', zero_division=0)
        rec = recall_score(y_true, y_pred, average='macro', zero_division=0)
        f1_score_val = f1_score(y_true, y_pred, average='macro', zero_division=0)
        precision.append(prec)
        recall.append(rec)
        f1.append(f1_score_val)
        total_uplink_airtime.append(uplink_airtime)
        round_completion_time.append(uplink_end_time[round])
        total_client_updates_delivered.append(total_updates_delivered)
        
        print(f"\nResults for Session {session_num}, Round {round}:\n\tSampled clients: {sampled_clients} \n\tGlobal update received by {updated_clients} \n\tLocal updates delivered by {local_updates_delivered} \n\tround_completion_time = {round_completion_time[round] } \n\tdownlink_airtime = {downlink_airtime[round]} \n\ttotal uplink_airtime = {total_uplink_airtime[round]} \n\tAccuracy {pred_accuracy} \n\tLoss {pred_loss} \n\tprecision_score = {prec} \n\trecall_score = {rec} \n\tF1_score = {f1_score_val}")
        
    return client_update_sizes, total_uplink_airtime, downlink_airtime, round_completion_time , total_client_updates_delivered

In [ ]:
#
# The Main Program: Federated Learning With Long Range (LoRa) Comunications
#

def main():

    
    global global_accuracy, global_loss, prediction_accuracy
    global prediction_loss, precision, recall, f1    
    global full_simulation
    global total_tx_all 
    global total_rx_all 
    global interferer_intensity, simulation_radius
    global mean_interf_interval, interf_payload_range, LoRa_channel_code, bandwidth, Ptx, path_loss_exp 
    global spreading_factor_FL, num_channels, sensitivity, cap_thresh, GtGr, d0, Lpld0

            
    all_prediction_accuracies = []
    all_airtime_uplink = []
    all_airtime_downlink = []
    all_completion_time = []
    all_client_updates_delivered = []
    execution_time = []

    # Number of FL sessions to simulate
    num_sessions = 20

    # Number of training rounds per session   
    rounds_per_session = 15
    
    # Number of clients
    num_clients = 20

    # Network radius in meters (clients will be randomly deployed within a circle of this radius, with the server at the origin)
    network_radius = 500

    # Spreading factor to be used for FL update transfers
    spreading_factor_FL = 9
    
    # Whether to quantize the parameter values, and if so, to how many bits (supports 1, 2, and 4)
    # If False, then 32-bit reprentation will be used
    use_quantization = True
    
    if use_quantization:
        quantization_bits = 4
    else:
        quantization_bits = 0

    # Whether to apply sparsity (set parameter values below a threshold to zero), and if so, what threshold to use
    use_sparsity = True
    if use_sparsity:
        sparsity_threshold = 0.001
    else:
        sparsity_threshold = 0

    # Whether to compress the updates using zlib compression prior to transmission
    apply_zlib = True

    # The class of LoRaWAN operation to be used (supports class B and C)
    LoRa_Class = 'B'
        
    # Ping slot periodicity (only for Class B)
    ping_slot_duration = 0.03
    
    # Sampling ratio (the fraction of clients chosen for update exchange in each round)
    sampling_ratio = 0.4
        
    # Rate of the FEC applied to the fragments
    FEC_rate = 0.5

    # Maximum permitted value for transmitter duty cycle (in percentage)
    duty_cycle_percentage = 1.0

    # No. of interferers per sq. meter (assumes a Poisson Point Process)
    interferer_intensity = 10.0**(-4.0)
    
    # Whether to actually simulate LoRa tranmissions or apply analytical approximations
    full_simulation = False

    # Mean of the time interval (in milliseconds) between two consecutive packets from an interferer
    mean_interf_interval = 60000 

    # Range of values for interfering frame payload (an interfering frame carries a payload that is chosen uniformly at random from within this range)             
    interf_payload_range = range(1,50)  

    # Channel coding rate for LoRa frames
    LoRa_channel_code = 1 

    # Transmission bandwidth (kHz)
    bandwidth = 125 

    # power of the transmitted signal in dBm
    Ptx = 14 

    # Path loss exponent
    path_loss_exp = 2.5 

    # Number of non overlapping frequency bands available to LoRa transmitters
    num_channels = 8 
    
    # LoRa receiver sensitivities (for spreading factors 7 through 12) 
    sensitivity = [-123, -126, -129, -132, -134.5, -137]
    
    # capture thresholds 
    cap_thresh = np.array([[1, -8, -9, -9, -9, -9],
                           [-11, 1, -11, -12, -13, -13],
                           [-15, -13, 1, -13, -14, -15],
                           [-19, -18, -17, 1, -17, -18],
                           [-22, -22, -21, -20, 1, -20],
                           [-25, -25, -25, -24, -23, 1]])
        
    # Product of tx and rx antenna gains (dB)  
    GtGr = 6

    # Reference distance (for LoRa signal strength computation)
    d0 = 40.0

    # Path loss (dB) at reference distance
    Lpld0 = 127.41

    # Matrices to store outputs
    all_prediction_accuracies = []
    all_airtime_uplink = []
    all_airtime_downlink = []
    all_completion_time = []
    all_client_updates_delivered = []
    execution_time = []

    # For keeping track of total transmissions and receptions (for debugging purposes)
    total_tx_all = 0
    total_rx_all = 0
    
    # Capture timestamp for the simulation (in IST)
    ist = pytz.timezone('Asia/Kolkata')
    simulation_timestamp = datetime.now(ist).strftime("%I:%M %p IST, %A, %B %d, %Y")
    
    trainloaders, valloaders, testloader, datasets = load_datasets(num_clients)
    
    for session in range(num_sessions):
        
        # Reset global variables
        global_accuracy = []
        global_loss = []
        prediction_accuracy = []
        prediction_loss = []
        precision = []
        recall = []
        f1 = []

        start_time = time.time()
                

        # Run simulation
        client_update_sizes, total_uplink_airtime, total_airtime_downlink, round_completion_time , client_updates_delivered  = start_simulation(session, total_rounds = rounds_per_session, num_clients = num_clients,
                                                                   network_radius = network_radius, duty_cycle_percentage = duty_cycle_percentage, sampling_ratio = sampling_ratio, use_quantization = use_quantization,use_sparsity = use_sparsity, sparsity_threshold = sparsity_threshold, apply_zlib = apply_zlib, quantization_bits = quantization_bits, ping_slot_duration = ping_slot_duration, spreading_factor_FL = spreading_factor_FL, FEC_rate = FEC_rate, LoRa_Class = LoRa_Class, trainloaders= trainloaders, testloader= testloader)
        # Collect results
        all_prediction_accuracies.append(prediction_accuracy)
        all_airtime_uplink.append(total_uplink_airtime)
        all_airtime_downlink.append(total_airtime_downlink)
        all_completion_time.append(round_completion_time )
        all_client_updates_delivered.append(client_updates_delivered)

        end_time = time.time()
        execution_time.append(end_time - start_time)
    
    plt.figure(figsize=(12,8))
    rounds= list(range(1, rounds_per_session + 1))

    for exp, prediction_accuracy in enumerate(all_prediction_accuracies):           
        plt.plot(rounds, prediction_accuracy, label=f'Session {exp+1}', marker= 'o')

    
    #Compute averages  
    average_accuracy = np.mean(all_prediction_accuracies, axis = 0)
    average_uplink_time = np.mean(all_airtime_uplink, axis = 0)
    average_downlink_time = np.mean(all_airtime_downlink, axis = 0)
    average_completion_time = np.mean(all_completion_time, axis = 0)    
    average_client_updates_delivered = np.mean(all_client_updates_delivered, axis = 0)
    average_simulation_time = np.mean(execution_time)
    
    # print averaged outputs
    print('\n average accuracy: ', average_accuracy)
    print('average completion time: ', average_completion_time)
    print('average uplink time: ', average_uplink_time)
    print('average downlink time: ', average_downlink_time)
    print('average client_updates delivered: ', average_client_updates_delivered)
    print('simulation time per session: ', average_simulation_time)
    
    # Write results to a file
    filename = "Output.txt"
    def write_results(filename, spreading_factor_FL, FEC_rate, average_accuracy, average_uplink_time, average_downlink_time, average_completion_time, average_client_updates_delivered):
        with open(filename, "a") as file:
            file.write(f"Simulation Run Timestamp: {simulation_timestamp}\n")
            file.write("=" * 36 + "\n")
            # Write header with parameters
            file.write("Federated Learning Simulation Results\n")
            file.write("=" * 36 + "\n")
            file.write(f"Spreading factor: {spreading_factor_FL}\n")
            file.write(f"FEC rate: {FEC_rate}\n")
            file.write(f"Number of FL sessions: {num_sessions}\n")
            file.write(f"Number of training rounds per experiment: {rounds_per_session}\n")
            file.write(f"Number of clients: {num_clients}\n")
            file.write(f"Network radius (in meters): {network_radius}\n")
            file.write(f"Interferer intensity: {interferer_intensity}\n")
            file.write(f"Mean interference interval (milliseconds): {mean_interf_interval}\n")
            file.write(f"Quantization bits: {quantization_bits}\n")
            file.write(f"Sparsity threshold: {sparsity_threshold}\n")
            file.write(f"LoRa Class of Operation: {LoRa_Class}\n")
            file.write(f"Ping slot periodicity (in milliseconds): {ping_slot_duration}\n")
            file.write(f"Sampling ratio: {sampling_ratio}\n")
            file.write(f"Transmitter duty-cycle percentage: {duty_cycle_percentage}\n\n")
            file.write(f"Full_simulation: {full_simulation}\n\n")
            
            # Write table header with fixed-width columns
            file.write("| {:<5} | {:>12} | {:>19} | {:>20} | {:>19} | {:>23} |\n".format(
                "Round", "Accuracy (%)", "Uplink Airtime (ms)", "Downlink Airtime (ms)", "Completion Time (ms)", "Client Updates Delivered"
            ))
            file.write("| {:<5} | {:>12} | {:>19} | {:>20} | {:>19} | {:>23} |\n".format(
                "-" * 5, "-" * 12, "-" * 19, "-" * 20, "-" * 19, "-" * 23
            ))
            
            # Write table rows
            for round_idx in range(len(average_accuracy)):
                file.write("| {:<5} | {:>12.2f} | {:>19.2f} | {:>20.2f} | {:>19.2f} | {:>23.2f} |\n".format(
                    round_idx + 1,
                    average_accuracy[round_idx] * 100,  # Convert to percentage
                    average_uplink_time[round_idx],
                    average_downlink_time[round_idx],
                    average_completion_time[round_idx],
                    average_client_updates_delivered[round_idx]
                ))
            
            # Add notes
            file.write("\nNotes:\n")
            file.write("- Accuracy: Percentage of correct predictions.\n")
            file.write("- Uplink Airtime: Time spent by clients in transmitting local updates (s).\n")
            file.write("- Downlink Airtime: Time spent by the server in transmitting the global update (s).\n")
            file.write("- Completion Time: Time elapsed till round completion (s).\n")
            file.write("- Client Updates Delivered: Average number of clients that successfully delivered their local update to the server.\n")
            file.write("\n")

    
    plt.plot(rounds, average_accuracy, label = 'Average Accuracy', color = 'black', linewidth = 2, linestyle ='--', marker= 'x')
    write_results(filename, spreading_factor_FL, FEC_rate, average_accuracy, average_uplink_time, average_downlink_time, average_completion_time, average_client_updates_delivered )

    filename="FedAvg_MNIST_LoRaWAN.png"
    plt.xlabel('Round')
    plt.ylabel('Prediction Accuracy')
    plt.title('Average Prediction accuracy vs rounds')
    plt.legend()
    plt.grid(True)
    plt.show
    plt.savefig(filename)

    print(f"Plot saved as {filename}")
    
    

if __name__ == "__main__":
    main()